In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
import yfinance as yf
from numpy.linalg import norm
import pandas as pd
import random
yf.pdr_override()
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import fastdtw


In [ ]:
nasdaq = pdr.get_data_yahoo('^IXIC')
freq = "D"


[*********************100%***********************]  1 of 1 completed


In [ ]:
nasdaq

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1971-02-05,100.000000,100.000000,100.000000,100.000000,100.000000,0
1971-02-08,100.839996,100.839996,100.839996,100.839996,100.839996,0
1971-02-09,100.760002,100.760002,100.760002,100.760002,100.760002,0
1971-02-10,100.690002,100.690002,100.690002,100.690002,100.690002,0
1971-02-11,101.449997,101.449997,101.449997,101.449997,101.449997,0
...,...,...,...,...,...,...
2023-05-01,12210.049805,12261.320312,12181.080078,12212.599609,12212.599609,5168430000
2023-05-02,12198.019531,12206.580078,12015.230469,12080.509766,12080.509766,5501410000
2023-05-03,12097.040039,12212.549805,12022.459961,12025.330078,12025.330078,5782160000


In [ ]:
nasdaq = nasdaq[['Adj Close', 'Volume']]
nasdaq = nasdaq.loc[nasdaq.index >= '2000-01-01']
nasdaq

,Adj Close,Volume
Date,,
2000-01-03,4131.149902,1510070000
2000-01-04,3901.689941,1511840000
2000-01-05,3877.540039,1735670000
2000-01-06,3727.129883,1598320000
2000-01-07,3882.620117,1634930000
...,...,...
2023-05-01,12212.599609,5168430000
2023-05-02,12080.509766,5501410000
2023-05-03,12025.330078,5782160000


In [ ]:
nasdaq['Adj Close'] = np.log(nasdaq['Adj Close'])

In [ ]:
window_sizes = [20, 40, 60, 80, 100,120]
for window in window_sizes:
    nasdaq[f'Adj Close_{window}'] = nasdaq['Adj Close'].diff().rolling(window=window).mean()
for window in window_sizes:
    nasdaq[f'Volume{window}'] = nasdaq['Volume'].rolling(window =window).mean()


In [ ]:
window_size = 20
rolling_data_adjclose = nasdaq[['Adj Close'] + [f'Adj Close_{window}' for window in window_sizes]].rolling(window_size)
rolling_data_volume = nasdaq[['Volume'] + [f'Volume{window}' for window in window_sizes]].rolling(window_size)

In [ ]:
def sum_abs_error(data):
    unseen_data = data.iloc[-1]
    rolling_data = data.iloc[:-1]
    abs_error = np.abs(rolling_data - unseen_data)
    sum_error = abs_error.sum()
    return sum_error
sum_abs_error_adj = rolling_data_adjclose.apply(sum_abs_error)
sum_abs_error_vol = rolling_data_volume.apply(sum_abs_error)

In [ ]:
sum_abs_error_adj

,Adj Close,Adj Close_20,Adj Close_40,Adj Close_60,Adj Close_80,Adj Close_100,Adj Close_120
Date,,,,,,,
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-05-01,0.220785,0.241495,0.188313,0.176991,0.320851,0.137745,0.184119
2023-05-02,0.134085,0.200952,0.189723,0.155435,0.329766,0.140087,0.186454
2023-05-03,0.160604,0.164727,0.194556,0.137785,0.330864,0.142941,0.187432


In [ ]:
sum_abs_error_vol

,Volume,Volume20,Volume40,Volume60,Volume80,Volume100,Volume120
Date,,,,,,,
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-05-01,8.414190e+09,3.122243e+09,516364750.0,2.595578e+09,7.081838e+08,6.127905e+08,5.987908e+07
2023-05-02,1.366528e+10,2.772906e+09,564953250.0,2.495464e+09,8.763735e+08,7.602802e+08,1.607359e+08
2023-05-03,1.779688e+10,3.036809e+09,623725000.0,2.190261e+09,1.006254e+09,1.015074e+09,2.570714e+08


In [ ]:
sum_abs_error_adj.mean()

Adj Close        0.606896
Adj Close_20     0.408396
Adj Close_40     0.305634
Adj Close_60     0.259199
Adj Close_80     0.226171
Adj Close_100    0.204132
Adj Close_120    0.192233
dtype: float64

In [ ]:
sum_abs_error_adj.std()

Adj Close        0.473729
Adj Close_20     0.348595
Adj Close_40     0.271725
Adj Close_60     0.214935
Adj Close_80     0.190746
Adj Close_100    0.175471
Adj Close_120    0.162344
dtype: float64